In [1]:
!pip install mediapipe opencv-python
!pip install tensorflow

  Using cached mediapipe-0.10.18-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached jax-0.4.36-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.36-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.0 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached ml_dtypes-0.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow
import os
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder,StandardScaler
from tqdm import tqdm
from pathlib import Path
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight

2024-12-09 17:43:36.450576: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 17:43:36.450953: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-09 17:43:36.452831: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-09 17:43:36.457810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733759016.466092  111642 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733759016.46

In [2]:
EXPECTED_KEYPOINTS = 200

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.3, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1733759024.291916  111642 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1733759024.321330  111773 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce RTX 4080/PCIe/SSE2


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733759024.337783  111745 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733759024.360761  111765 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
def get_bboxes(image): #using media pipe since im just testing
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    bounding_boxes = {"left_hand": None, "right_hand": None}

    if results.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            x_min, y_min = float('inf'), float('inf')
            x_max, y_max = float('-inf'), float('-inf')

            for landmark in hand_landmarks.landmark:
                x, y = landmark.x * image.shape[1], landmark.y * image.shape[0]
                x_min, y_min = min(x_min, x), min(y_min, y)
                x_max, y_max = max(x_max, x), max(y_max, y)

            hand_label = "left_hand" if idx == 0 else "right_hand"
            bounding_boxes[hand_label] = (int(x_min), int(y_min), int(x_max), int(y_max))

    return bounding_boxes

In [5]:
def detect_keypoints(image, max_keypoints=200, target_length=None):
    gray_hand = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_hand, None)

    if descriptors is None:
        descriptors = np.zeros((len(keypoints), 128))  # Ensure descriptors match number of keypoints

    keypoints = sorted(keypoints, key=lambda kp: kp.response, reverse=True)

    # Ensure exactly max_keypoints keypoints and descriptors
    keypoints = keypoints[:max_keypoints]
    descriptors = descriptors[:max_keypoints]  # Limit descriptors to match keypoints

    keypoint_locations = [keypoint.pt for keypoint in keypoints]

    # Pad with dummy keypoints and descriptors if necessary
    if len(keypoints) < max_keypoints:
        num_keypoints_to_add = max_keypoints - len(keypoints)

        # Create dummy keypoints (pts = (0, 0)) and dummy descriptors (zeros)
        dummy_keypoints = [(0.0, 0.0)] * num_keypoints_to_add  # List of (0, 0) tuples
        dummy_descriptors = np.zeros((num_keypoints_to_add, 128))  # Empty descriptors for dummy keypoints

        keypoint_locations.extend(dummy_keypoints)  # Add dummy keypoints
        descriptors = np.vstack((descriptors, dummy_descriptors))  # Stack dummy descriptors

    return keypoint_locations, descriptors



In [6]:
def display_keypoints(image, bounding_boxes):

    for label, coords in bounding_boxes.items():
        if coords:
            x_min, y_min, x_max, y_max = coords
            cropped_hand = image[y_min:y_max, x_min:x_max]

            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            keypoints = detect_keypoints(cropped_hand)

            for keypoint in keypoints:
                x, y = keypoint
                cv2.circle(cropped_hand, (int(x), int(y)), 5, (0, 0, 255), -1)
    cv2_imshow(image)


In [ ]:
input_image = cv2.imread('test.jpg')

In [163]:
bounding_boxes = get_bboxes(input_image)

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [164]:
display_keypoints(input_image, bounding_boxes)

NameError: name 'bounding_boxes' is not defined

In [7]:
def generate_dummy_keypoints_and_descriptors(max_keypoints=200):
    # Create dummy keypoints as (0.0, 0.0) pairs
    dummy_keypoints = [(0.0, 0.0)] * max_keypoints

    # Create dummy descriptors as arrays of zeros (shape: (max_keypoints, 128))
    dummy_descriptors = np.zeros((max_keypoints, 128))

    return dummy_keypoints, dummy_descriptors

In [8]:
MAX_HANDS =2
def load_data_from_directory(root_directory, total_images, data_fraction=1.0):
    """
    Load data from the given directory, with an option to limit the amount of data by a fraction.

    Parameters:
    - root_directory: Path to the root directory containing image files.
    - total_images: Total number of images available in the directory.
    - data_fraction: A decimal value representing the fraction of the data to load (e.g., 0.8 for 80%).

    Returns:
    - X: List of keypoints for each image.
    - y: List of corresponding labels for each image.
    """
    X = []
    y = []

    # Calculate the number of images to process based on the given fraction
    max_images_to_process = int(total_images * data_fraction)
    processed_images = 0

    with tqdm(total=max_images_to_process, desc="Processing Images", unit="image") as pbar:
        for label in os.listdir(root_directory):
            label_path = os.path.join(root_directory, label)

            if os.path.isdir(label_path):
                for subdir, _, files in os.walk(label_path):
                    for file in files:
                        if file.endswith(('.jpg', '.png', '.jpeg')):
                            if processed_images >= max_images_to_process:
                              return np.array(X), np.array(y)
                            try:
                              image_path = os.path.join(subdir, file)
                              image = cv2.imread(image_path)
                              bounding_boxes = get_bboxes(image)
                              all_keypoints = []
                              hands_detected = False
                              for hand_label, bounding_box in bounding_boxes.items():
                                  if bounding_box:  # If bounding box exists
                                      hands_detected = True
                                      x_min, y_min, x_max, y_max = bounding_box
                                      cropped_hand = image[y_min:y_max, x_min:x_max]

                                      keypoints, descriptors = detect_keypoints(cropped_hand)
                                      all_keypoints = keypoints
                              if hands_detected:
                                  num_keypoints = len(all_keypoints)
                                  target_length = EXPECTED_KEYPOINTS * MAX_HANDS
                                  if num_keypoints == 0:
                                      pbar.update(1)
                                      continue
                                  elif num_keypoints < target_length:
                                      dummy_keypoints, dummy_descriptors = generate_dummy_keypoints_and_descriptors(target_length - num_keypoints)
                                      all_keypoints.extend(dummy_keypoints)
                                      descriptors = np.vstack([descriptors, dummy_descriptors])


                                  elif num_keypoints > target_length:
                                    print("MoORE")
                                      # all_keypoints = all_keypoints[:target_length]
                                      # for descriptor in descriptors:
                                      #     descriptor = descriptor[:target_length]
                                      # keypoint_descriptors = np.concatenate([np.array(all_keypoints).flatten(), descriptors.flatten()])
                                  keypoint_descriptors = np.concatenate([np.array(all_keypoints).flatten(), descriptors.flatten()])
                                  X.append(keypoint_descriptors)
                                  y.append(label)
                            except:
                              pass

                            processed_images += 1
                            pbar.update(1)

                    if processed_images >= max_images_to_process:
                        break  # Break the outer loop if we have processed enough images

            if processed_images >= max_images_to_process:
                break  # Break the main loop if we have processed enough images

    return np.array(X), np.array(y)


In [9]:
 # Normalize the landmarks to [0, 1]
def preprocess_landmarks(keypoints, image_width, image_height):
    processed_landmarks = []
    for i in keypoints:
        x, y = i
        processed_landmarks.append(x / image_width)
        processed_landmarks.append(y / image_height)

    return np.array(processed_landmarks)

In [10]:
def build_fc_model(input_shape):
    model = models.Sequential()

    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(len(np.unique(y)), activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Learning rate adjustment
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


In [11]:
def train_model(X, y, epochs=10, batch_size=32):
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    X = X.reshape(X.shape[0], -1)
    scaler = StandardScaler()
    X_encoded = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.1, random_state=42)

    model = build_fc_model(input_shape=(X_train.shape[1], ))

    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test)
    )

    return model, scaler, label_encoder, history


In [12]:
"""
The way this function works is as follows

1- loop over a directory
2- for each item in the directory check if its an image
3- if its an image take the first item after splitting by underscore
4- That item is the label, for example hello_4.jpg will be labeled as hello, so make sure the images should be stored that way
5- Bounding boxes are extracted
6- For each bounding box we get the key points
7- If exists we then normaize it using the preprocess landmarks function
8- We add it them to the x and y arrays that will be used to train the model
"""
def predict_sign_language(model, label_encoder, scaler, image):
    bounding_boxes = get_bboxes(image)
    all_keypoints = []  # This will hold the keypoints for all detected hands
    hands_detected = False
    for _, bounding_box in bounding_boxes.items():
        if bounding_box:
            hands_detected = True
            x_min, y_min, x_max, y_max = bounding_box
            cropped_hand = image[y_min:y_max, x_min:x_max]

            keypoints = detect_keypoints(cropped_hand)

            if keypoints:
                processed_keypoints = preprocess_landmarks(keypoints, image.shape[1], image.shape[0])
                all_keypoints.extend(processed_keypoints)
    if hands_detected:
        num_keypoints = len(all_keypoints)
        if num_keypoints == 0:
          return None
        elif num_keypoints < EXPECTED_KEYPOINTS * MAX_HANDS:
            all_keypoints.extend([0] * (EXPECTED_KEYPOINTS * MAX_HANDS - num_keypoints))
        elif num_keypoints > EXPECTED_KEYPOINTS * MAX_HANDS:
            all_keypoints = all_keypoints[:EXPECTED_KEYPOINTS * MAX_HANDS]
        processed_keypoints = np.array(all_keypoints).reshape(1, -1, 1)
        processed_keypoints = scaler.transform(processed_keypoints)
        prediction = model.predict(processed_keypoints)
        predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])

        return predicted_label[0]

    return None


In [13]:
def save_data_to_file(X, y, file_name='data.npz'):
    np.savez(file_name, X=X, y=y)
    print(f"Data saved to {file_name}")

In [14]:
def load_data_from_file(file_name='data.npz'):
    data = np.load(file_name)
    X = data['X']
    y = data['y']
    print(f"Data loaded from {file_name}")
    return X, y

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
images_dir = '/content/drive/MyDrive/compFrames/'


In [19]:
X, y = load_data_from_directory(images_dir,165000,0.1)
save_data_to_file(X, y, file_name='data_sift.npz')

Processing Images: 100%|██████████| 16500/16500 [2:32:27<00:00,  1.80image/s]


Data saved to data_sift.npz


In [15]:
X, y = load_data_from_file('data_sift.npz')

Data loaded from data_sift.npz


In [20]:
save_data_to_file(X, y, file_name='/content/drive/MyDrive/data_sift.npz')

Data saved to /content/drive/MyDrive/data_sift.npz


In [16]:
print(np.unique(y))

['above' 'accomplish' 'adopt' 'advantage' 'alphabet' 'anniversary'
 'another' 'appropriate' 'arrogant' 'artist' 'background' 'bee' 'behavior'
 'bell' 'birth' 'bless' 'blood' 'boots' 'brave' 'breathe' 'bright' 'bull'
 'butterfly' 'card' 'category' 'catholic' 'cent' 'chemistry' 'cherry'
 'china' 'chop' 'christian' 'cigarette' 'clever' 'clown' 'coach' 'coat'
 'cochlear implant' 'coconut' 'common' 'commute' 'control' 'count'
 'culture' 'daily' 'defend' 'degree' 'demonstrate' 'department' 'die'
 'dig' 'dinner' 'dinosaur' 'diploma' 'director' 'disconnect' 'discover'
 "don't want" 'drum' 'each' 'educate' 'electrician' 'empty' 'england'
 'establish' 'excuse' 'eyeglasses' 'faculty' 'fall in love' 'famous'
 'farm' 'fence' 'fix' 'flexible' 'flirt' 'flood' 'fork' 'four' 'fun'
 'funeral' 'furniture' 'gallaudet' 'general' 'ghost' 'give up' 'glass'
 'gorilla' 'gray' 'guide' 'habit' 'health' 'hearing aid' 'heaven' 'heavy'
 'helmet' 'hide' 'high school' 'hockey' 'hold' 'honest' 'hug' 'hunt'
 'image' 'i

In [17]:
model,scaler,label_encoder,history = train_model(X, y,epochs=50)

/home/g7/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
W0000 00:00:1733759079.557999  111642 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 40s 151ms/step - accuracy: 0.0089 - loss: 7.6525 - val_accuracy: 0.0131 - val_loss: 6.7722
Epoch 2/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 39s 151ms/step - accuracy: 0.0169 - loss: 6.8591 - val_accuracy: 0.0197 - val_loss: 6.7964
Epoch 3/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 39s 149ms/step - accuracy: 0.0336 - loss: 6.6946 - val_accuracy: 0.0306 - val_loss: 6.7763
Epoch 4/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 39s 150ms/step - accuracy: 0.0514 - loss: 6.5485 - val_accuracy: 0.0328 - val_loss: 6.9329
Epoch 5/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 39s 149ms/step - accuracy: 0.0682 - loss: 6.4727 - val_accuracy: 0.0284 - val_loss: 6.8784
Epoch 6/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 39s 149ms/step - accuracy: 0.0989 - loss: 6.3410 - val_accuracy: 0.0438 - val_loss: 6.9273
Epoch 7/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 39s 150ms/step - accuracy: 0.1192 - loss: 6.2594 - val_accuracy: 0.0361 - val_loss: 7.2161
Epoch 8/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 39s 151ms/step - accuracy: 0.1309 - loss: 6

In [ ]:
image = cv2.imread('test_image.jpg')

In [ ]:
###seems to need reset after running tensorflow probably something with the cache
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [ ]:
bounding_boxes = get_bboxes(image)
print(bounding_boxes)

{'left_hand': (488, 447, 703, 719), 'right_hand': None}


In [ ]:
predicted_sign = predict_sign_language(model,label_encoder, image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


In [ ]:
print(f"Predicted sign language gesture: {predicted_sign}")

Predicted sign language gesture: thanks
